In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [2]:
rating_df = pd.read_csv('C:/Users/kiddc/jupyter_資料/專題演算法-restaurant.csv')
rating_df.head()

,Unnamed: 0,res_id,res_name,address,score,price,content,article_url,image_url,tag
0,0,20000001,紅磡 新飲茶,臺北市中山區林森北路263號,4.3,600.0,"['台北老字號港式茶餐廳', '上百道高人氣港點', '粵菜', '品嚐到最正宗港式的料理風...",https://ifoodie.tw/restaurant/59c2ae542756dd67...,https://lh3.googleusercontent.com/46_fNhwBpOwH...,NaN
1,1,20000002,味津烤鴨,臺北市大同區民族西路223之17號,4.4,200.0,"['網路推爆櫻桃鴨專賣店', '片皮鴨', '鹹酥', '三杯', '醬燒雙重享受超難打通的...",https://ifoodie.tw/restaurant/5b2e8c622756dd1d...,https://lh3.googleusercontent.com/ooeYghZcULwO...,NaN
2,2,20000003,萬麗軒 Wan Li Restaurant - 台北士林萬麗酒店,臺北市士林區中山北路五段470巷8號1樓,4.6,NaN,"['士林美食', '台北士林萬麗酒店', '士林廚房經典海陸套餐', '士林官邸旁', '捷...",https://ifoodie.tw/restaurant/607305e97cc7fc03...,https://lh3.googleusercontent.com/UMPfucoZtCNX...,NaN
3,3,20000004,款待 手路菜,臺北市中山區中山北路一段53巷32號,4.2,500.0,"['款待台菜', '台菜吃到飽推薦', '菜單', '價錢', '款待', '手路菜', '...",https://ifoodie.tw/restaurant/59316bdff5246848...,https://lh3.googleusercontent.com/o5YghJN_PKQS...,NaN
4,4,20000005,飯BAR 內湖旗艦店,臺北市內湖區內湖路一段360巷2號,4.3,500.0,"['內湖旗艦店', '中式料理也可以吃得很有儀式感', '必點花雕遇到雞', '美人茶遇東坡...",https://ifoodie.tw/restaurant/559d549ac03a103e...,https://lh3.googleusercontent.com/ZVtC9unIUUsv...,NaN


In [3]:
# 移除 CONTENT 欄位
rating_df.drop(['article_url','image_url','tag',], axis = 1, inplace = True)

In [4]:
location1 = {
    '中正區': '中正區',  '萬華區': '中正區',  '大同區': '大同區',  '中山區': '中山區',  '松山區': '松山區',
    '大安區': '大安區',  '信義區': '信義區',  '內湖區': '內湖區',  '南港區':'南港區',  '士林區':'士林區',
    '北投區': '北投區', '萬里區': '萬里區', '金山區': '金山區', '板橋區': '板橋區', '汐止區': '汐止區',
    '深坑區': '深坑區', '石碇區': '石碇區', '瑞芳區': '瑞芳區', '平溪區':'平溪區', '雙溪區': '雙溪區', 
    '貢寮區': '貢寮區', '新店區': '新店區', '坪林區': '坪林區', '烏來區': '烏來區', '永和區': '永和區',
    '中和區':'中和區', '土城區': '土城區', '三峽區': '三峽區', '樹林區': '樹林區', '鶯歌區':'鶯歌區',
    '三重區': '三重區', '新莊區': '新莊區', '泰山區': '泰山區', '林口區': '林口區', '蘆洲區': '蘆洲區',
    '五股區': '五股區', '八里區': '八里區', '淡水區': '淡水區', '三芝區': '三芝區', '石門區': '石門區',
    '宜蘭市': '宜蘭市', '羅東鎮': '羅東鎮', '蘇澳鎮': '蘇澳鎮', '頭城鎮': '頭城鎮', '礁溪鄉': '礁溪鄉',
    '壯圍鄉': '壯圍鄉', '員山鄉': '員山鄉', '冬山鄉': '冬山鄉', '五結鄉': '五結鄉', '三星鄉': '三星鄉',
    '大同鄉': '大同鄉', '南澳鄉': '南澳鄉'
    } 

def location_clean(series):
    global location1
    try:
        for key, value in location1.items():
            if key in series:
#                 print(key, '\t',series)
                return value
            else:
                continue
    except TypeError:
        return None

rating_df['clean_add'] =rating_df['address'].apply(lambda x: location_clean(x))
rating_df[['clean_add']]

,clean_add
0,中山區
1,大同區
2,士林區
3,中山區
4,內湖區
...,...
33444,None
33445,信義區
33446,大安區
33447,中正區


In [5]:
rating_df['res_name']=rating_df['res_name'].str.split('完整食記請點這裡',expand=True)[0]

In [6]:
religion = [
    '中正區',  '萬華區',  '大同區', 
    '內湖區',  '南港區',  '士林區','文山區'
    '北投區', '萬里區', '金山區', '板橋區', '汐止區',
    '深坑區', '石碇區', '瑞芳區', '平溪區', '雙溪區', 
    '貢寮區', '新店區', '坪林區', '烏來區', '永和區',
    '中和區', '土城區', '三峽區', '樹林區', '鶯歌區',
    '三重區', '新莊區', '泰山區', '林口區', '蘆洲區',
    '五股區', '八里區', '淡水區', '三芝區', '石門區',
    '宜蘭市', '羅東鎮', '蘇澳鎮', '頭城鎮', '礁溪鄉',
    '壯圍鄉', '員山鄉', '冬山鄉', '五結鄉', '三星鄉',
    '大同鄉', '南澳鄉'
            ]

In [7]:
import random

In [8]:
rating_df['clean_add']=rating_df['clean_add'].fillna(random.choice(religion))
rating_df

,Unnamed: 0,res_id,res_name,address,score,price,content,clean_add
0,0,20000001,紅磡 新飲茶,臺北市中山區林森北路263號,4.3,600.0,"['台北老字號港式茶餐廳', '上百道高人氣港點', '粵菜', '品嚐到最正宗港式的料理風...",中山區
1,1,20000002,味津烤鴨,臺北市大同區民族西路223之17號,4.4,200.0,"['網路推爆櫻桃鴨專賣店', '片皮鴨', '鹹酥', '三杯', '醬燒雙重享受超難打通的...",大同區
2,2,20000003,萬麗軒 Wan Li Restaurant - 台北士林萬麗酒店,臺北市士林區中山北路五段470巷8號1樓,4.6,NaN,"['士林美食', '台北士林萬麗酒店', '士林廚房經典海陸套餐', '士林官邸旁', '捷...",士林區
3,3,20000004,款待 手路菜,臺北市中山區中山北路一段53巷32號,4.2,500.0,"['款待台菜', '台菜吃到飽推薦', '菜單', '價錢', '款待', '手路菜', '...",中山區
4,4,20000005,飯BAR 內湖旗艦店,臺北市內湖區內湖路一段360巷2號,4.3,500.0,"['內湖旗艦店', '中式料理也可以吃得很有儀式感', '必點花雕遇到雞', '美人茶遇東坡...",內湖區
...,...,...,...,...,...,...,...,...
33444,33445,20033446,老柴咖啡館,台北市羅斯福路四段150號,3.8,100.0,"['餐廳名稱：老柴咖啡館', '消費時間：2013年/8月', '地址：新北市三峽區大觀路1...",五結鄉
33445,33446,20033447,舒果(板橋雙十店),台北市信義區松壽路12號3F-01,3.8,398.0,"['餐廳名稱：舒果(板橋雙十店)', '消費時間：2013年/8月', '地址：新北市板橋區...",信義區
33446,33447,20033448,義想義大利麵,台北市大安區光復南路102號1樓,3.1,100.0,"['餐廳名稱：義想義大利麵', '消費時間：2013年/8月', '地址:新北市永和區文化路...",大安區
33447,33448,20033449,楔子咖啡館,台北市萬華區昆明街214號之1/新北市瑞芳區豎崎路5號,3.0,NaN,"['餐廳名稱：楔子咖啡館', '消費時間：2013/08', '地址：新北市新莊區公園路16...",中正區


In [12]:
rating_df

,Unnamed: 0,res_id,res_name,address,score,price,content,clean_add
0,0,20000001,紅磡 新飲茶,臺北市中山區林森北路263號,4.3,600.0,"['台北老字號港式茶餐廳', '上百道高人氣港點', '粵菜', '品嚐到最正宗港式的料理風...",中山區
1,1,20000002,味津烤鴨,臺北市大同區民族西路223之17號,4.4,200.0,"['網路推爆櫻桃鴨專賣店', '片皮鴨', '鹹酥', '三杯', '醬燒雙重享受超難打通的...",大同區
2,2,20000003,萬麗軒 Wan Li Restaurant - 台北士林萬麗酒店,臺北市士林區中山北路五段470巷8號1樓,4.6,NaN,"['士林美食', '台北士林萬麗酒店', '士林廚房經典海陸套餐', '士林官邸旁', '捷...",士林區
3,3,20000004,款待 手路菜,臺北市中山區中山北路一段53巷32號,4.2,500.0,"['款待台菜', '台菜吃到飽推薦', '菜單', '價錢', '款待', '手路菜', '...",中山區
4,4,20000005,飯BAR 內湖旗艦店,臺北市內湖區內湖路一段360巷2號,4.3,500.0,"['內湖旗艦店', '中式料理也可以吃得很有儀式感', '必點花雕遇到雞', '美人茶遇東坡...",內湖區
...,...,...,...,...,...,...,...,...
33444,33445,20033446,老柴咖啡館,台北市羅斯福路四段150號,3.8,100.0,"['餐廳名稱：老柴咖啡館', '消費時間：2013年/8月', '地址：新北市三峽區大觀路1...",文山區
33445,33446,20033447,舒果(板橋雙十店),台北市信義區松壽路12號3F-01,3.8,398.0,"['餐廳名稱：舒果(板橋雙十店)', '消費時間：2013年/8月', '地址：新北市板橋區...",信義區
33446,33447,20033448,義想義大利麵,台北市大安區光復南路102號1樓,3.1,100.0,"['餐廳名稱：義想義大利麵', '消費時間：2013年/8月', '地址:新北市永和區文化路...",大安區
33447,33448,20033449,楔子咖啡館,台北市萬華區昆明街214號之1/新北市瑞芳區豎崎路5號,3.0,NaN,"['餐廳名稱：楔子咖啡館', '消費時間：2013/08', '地址：新北市新莊區公園路16...",萬華區


In [10]:
rating_df.iloc[33444,7]="三峽區"
rating_df

,Unnamed: 0,res_id,res_name,address,score,price,content,clean_add
0,0,20000001,紅磡 新飲茶,臺北市中山區林森北路263號,4.3,600.0,"['台北老字號港式茶餐廳', '上百道高人氣港點', '粵菜', '品嚐到最正宗港式的料理風...",中山區
1,1,20000002,味津烤鴨,臺北市大同區民族西路223之17號,4.4,200.0,"['網路推爆櫻桃鴨專賣店', '片皮鴨', '鹹酥', '三杯', '醬燒雙重享受超難打通的...",大同區
2,2,20000003,萬麗軒 Wan Li Restaurant - 台北士林萬麗酒店,臺北市士林區中山北路五段470巷8號1樓,4.6,NaN,"['士林美食', '台北士林萬麗酒店', '士林廚房經典海陸套餐', '士林官邸旁', '捷...",士林區
3,3,20000004,款待 手路菜,臺北市中山區中山北路一段53巷32號,4.2,500.0,"['款待台菜', '台菜吃到飽推薦', '菜單', '價錢', '款待', '手路菜', '...",中山區
4,4,20000005,飯BAR 內湖旗艦店,臺北市內湖區內湖路一段360巷2號,4.3,500.0,"['內湖旗艦店', '中式料理也可以吃得很有儀式感', '必點花雕遇到雞', '美人茶遇東坡...",內湖區
...,...,...,...,...,...,...,...,...
33444,33445,20033446,老柴咖啡館,台北市羅斯福路四段150號,3.8,100.0,"['餐廳名稱：老柴咖啡館', '消費時間：2013年/8月', '地址：新北市三峽區大觀路1...",三峽區
33445,33446,20033447,舒果(板橋雙十店),台北市信義區松壽路12號3F-01,3.8,398.0,"['餐廳名稱：舒果(板橋雙十店)', '消費時間：2013年/8月', '地址：新北市板橋區...",信義區
33446,33447,20033448,義想義大利麵,台北市大安區光復南路102號1樓,3.1,100.0,"['餐廳名稱：義想義大利麵', '消費時間：2013年/8月', '地址:新北市永和區文化路...",大安區
33447,33448,20033449,楔子咖啡館,台北市萬華區昆明街214號之1/新北市瑞芳區豎崎路5號,3.0,NaN,"['餐廳名稱：楔子咖啡館', '消費時間：2013/08', '地址：新北市新莊區公園路16...",中正區


In [11]:
rating_df['score']=rating_df['score'].fillna(rating_df['score'].mean()) 
rating_df['score']

0        4.3
1        4.4
2        4.6
3        4.2
4        4.3
        ... 
33444    3.8
33445    3.8
33446    3.1
33447    3.0
33448    2.3
Name: score, Length: 33449, dtype: float64

In [12]:
rating_df

,Unnamed: 0,res_id,res_name,address,score,price,content,clean_add
0,0,20000001,紅磡 新飲茶,臺北市中山區林森北路263號,4.3,600.0,"['台北老字號港式茶餐廳', '上百道高人氣港點', '粵菜', '品嚐到最正宗港式的料理風...",中山區
1,1,20000002,味津烤鴨,臺北市大同區民族西路223之17號,4.4,200.0,"['網路推爆櫻桃鴨專賣店', '片皮鴨', '鹹酥', '三杯', '醬燒雙重享受超難打通的...",大同區
2,2,20000003,萬麗軒 Wan Li Restaurant - 台北士林萬麗酒店,臺北市士林區中山北路五段470巷8號1樓,4.6,NaN,"['士林美食', '台北士林萬麗酒店', '士林廚房經典海陸套餐', '士林官邸旁', '捷...",士林區
3,3,20000004,款待 手路菜,臺北市中山區中山北路一段53巷32號,4.2,500.0,"['款待台菜', '台菜吃到飽推薦', '菜單', '價錢', '款待', '手路菜', '...",中山區
4,4,20000005,飯BAR 內湖旗艦店,臺北市內湖區內湖路一段360巷2號,4.3,500.0,"['內湖旗艦店', '中式料理也可以吃得很有儀式感', '必點花雕遇到雞', '美人茶遇東坡...",內湖區
...,...,...,...,...,...,...,...,...
33444,33445,20033446,老柴咖啡館,台北市羅斯福路四段150號,3.8,100.0,"['餐廳名稱：老柴咖啡館', '消費時間：2013年/8月', '地址：新北市三峽區大觀路1...",三峽區
33445,33446,20033447,舒果(板橋雙十店),台北市信義區松壽路12號3F-01,3.8,398.0,"['餐廳名稱：舒果(板橋雙十店)', '消費時間：2013年/8月', '地址：新北市板橋區...",信義區
33446,33447,20033448,義想義大利麵,台北市大安區光復南路102號1樓,3.1,100.0,"['餐廳名稱：義想義大利麵', '消費時間：2013年/8月', '地址:新北市永和區文化路...",大安區
33447,33448,20033449,楔子咖啡館,台北市萬華區昆明街214號之1/新北市瑞芳區豎崎路5號,3.0,NaN,"['餐廳名稱：楔子咖啡館', '消費時間：2013/08', '地址：新北市新莊區公園路16...",中正區


In [21]:
location = {'文山區':0,
    '中正區': 1,  '萬華區': 2,  '大同區': 3,  '中山區': 4,  '松山區': 5,
    '大安區': 6,  '信義區': 7,  '內湖區': 8,  '南港區': 9,  '士林區': 10,
    '北投區': 11, '萬里區': 12, '金山區': 13, '板橋區': 14, '汐止區': 15,
    '深坑區': 16, '石碇區': 17, '瑞芳區': 18, '平溪區': 19, '雙溪區': 20, 
    '貢寮區': 21, '新店區': 22, '坪林區': 23, '烏來區': 24, '永和區': 25,
    '中和區': 26, '土城區': 27, '三峽區': 28, '樹林區': 29, '鶯歌區': 30,
    '三重區': 31, '新莊區': 32, '泰山區': 33, '林口區': 34, '蘆洲區': 35,
    '五股區': 36, '八里區': 37, '淡水區': 38, '三芝區': 39, '石門區': 40,
    '宜蘭市': 41, '羅東鎮': 42, '蘇澳鎮': 42, '頭城鎮': 44, '礁溪鄉': 45,
    '壯圍鄉': 46, '員山鄉': 47, '冬山鄉': 48, '五結鄉': 49, '三星鄉': 50,
    '大同鄉': 51, '南澳鄉': 52
    } 

def addr_clean(series):
    global location
    try:
        for key, value in location.items():
            if key in series:
#                 print(key, '\t',series)
                return value
            else:
                continue
    except TypeError:
        return None

rating_df['clean_add_num'] = rating_df['clean_add'].apply(lambda x: addr_clean(x))
rating_df[['clean_add_num']]

,clean_add_num
0,4
1,3
3,4
4,8
5,1
...,...
33442,49
33444,28
33445,7
33446,6


In [23]:
# clean_add_num 歸類成新的clean_add_num_index
import statsmodels.api as sm
rating_df['clean_add_num_index'] = pd.Categorical(rating_df.clean_add).codes   
rating_df.head()

,res_id,res_name,address,score,price,content,clean_add,clean_add_num_index,clean_add_num
0,20000001,紅磡 新飲茶,臺北市中山區林森北路263號,4.3,600.0,"['台北老字號港式茶餐廳', '上百道高人氣港點', '粵菜', '品嚐到最正宗港式的料理風...",中山區,5,4
1,20000002,味津烤鴨,臺北市大同區民族西路223之17號,4.4,200.0,"['網路推爆櫻桃鴨專賣店', '片皮鴨', '鹹酥', '三杯', '醬燒雙重享受超難打通的...",大同區,20,3
3,20000004,款待 手路菜,臺北市中山區中山北路一段53巷32號,4.2,500.0,"['款待台菜', '台菜吃到飽推薦', '菜單', '價錢', '款待', '手路菜', '...",中山區,5,4
4,20000005,飯BAR 內湖旗艦店,臺北市內湖區內湖路一段360巷2號,4.3,500.0,"['內湖旗艦店', '中式料理也可以吃得很有儀式感', '必點花雕遇到雞', '美人茶遇東坡...",內湖區,10,8
5,20000006,黔園川菜老店,臺北市萬華區康定路25巷39號,4.0,500.0,"['黔園川菜老店', '捷運西門町站', '西門町60年老店熄燈號', '一代青春人的回憶'...",中正區,6,1


In [15]:
rating_df=rating_df.dropna(axis=0)
rating_df.isna().any()

Unnamed: 0             False
res_id                 False
res_name               False
address                False
score                  False
price                  False
content                False
clean_add              False
clean_add_num_index    False
dtype: bool

In [16]:
rating_df=rating_df.drop(['Unnamed: 0'],axis=1)

In [24]:
rating_df

,res_id,res_name,address,score,price,content,clean_add,clean_add_num_index,clean_add_num
0,20000001,紅磡 新飲茶,臺北市中山區林森北路263號,4.3,600.0,"['台北老字號港式茶餐廳', '上百道高人氣港點', '粵菜', '品嚐到最正宗港式的料理風...",中山區,5,4
1,20000002,味津烤鴨,臺北市大同區民族西路223之17號,4.4,200.0,"['網路推爆櫻桃鴨專賣店', '片皮鴨', '鹹酥', '三杯', '醬燒雙重享受超難打通的...",大同區,20,3
3,20000004,款待 手路菜,臺北市中山區中山北路一段53巷32號,4.2,500.0,"['款待台菜', '台菜吃到飽推薦', '菜單', '價錢', '款待', '手路菜', '...",中山區,5,4
4,20000005,飯BAR 內湖旗艦店,臺北市內湖區內湖路一段360巷2號,4.3,500.0,"['內湖旗艦店', '中式料理也可以吃得很有儀式感', '必點花雕遇到雞', '美人茶遇東坡...",內湖區,10,8
5,20000006,黔園川菜老店,臺北市萬華區康定路25巷39號,4.0,500.0,"['黔園川菜老店', '捷運西門町站', '西門町60年老店熄燈號', '一代青春人的回憶'...",中正區,6,1
...,...,...,...,...,...,...,...,...,...
33442,20033444,皇后鎮森林,台北市市民大道四段136-3號,3.5,450.0,"['餐廳名稱：皇后鎮森林', '消費時間：2013年/8月', '地址：台北縣三峽鎮竹崙里竹...",五結鄉,7,49
33444,20033446,老柴咖啡館,台北市羅斯福路四段150號,3.8,100.0,"['餐廳名稱：老柴咖啡館', '消費時間：2013年/8月', '地址：新北市三峽區大觀路1...",三峽區,0,28
33445,20033447,舒果(板橋雙十店),台北市信義區松壽路12號3F-01,3.8,398.0,"['餐廳名稱：舒果(板橋雙十店)', '消費時間：2013年/8月', '地址：新北市板橋區...",信義區,9,7
33446,20033448,義想義大利麵,台北市大安區光復南路102號1樓,3.1,100.0,"['餐廳名稱：義想義大利麵', '消費時間：2013年/8月', '地址:新北市永和區文化路...",大安區,22,6


In [25]:
restaurant_df = rating_df.pivot_table(index = 'res_name',columns = 'clean_add_num_index',values = 'clean_add_num')
restaurant_df.tail(500)

clean_add_num_index,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
res_name,,,,,,,,,,,,,,,,,,,,,
鷹流東京醬油拉麵-蘭丸拉麵,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
鷹流東京醬油拉麵【蘭丸ranmaru】中山店,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
鷹流東京醬油拉麵蘭丸,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
鷹流東京醬油拉麵蘭丸 中山店,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
鷹流東京醬油拉麵蘭丸(中山店),NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ＱＱ食堂,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ＲＡＷ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ｗoobar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
missing_value = restaurant_df.isna().sum()
missing_value

clean_add_num_index
0     20763
1     20805
2     20799
3     20501
4     20547
5     18066
6     18644
7     18352
8     20806
9     18841
10    20160
11    20809
12    20764
13    20515
14    20534
15    20785
16    20745
17    20807
18    20126
19    20804
20    19917
21    20809
22    16855
23    20622
24    20810
25    20651
26    20510
27    19477
28    20208
29    20695
30    20777
31    20536
32    20753
33    20801
34    20687
35    20801
36    20808
37    20787
38    20810
39    20803
40    20702
41    20650
42    20808
43    20684
44    20778
45    20804
46    20793
47    20767
48    20795
dtype: int64

In [27]:
rows = restaurant_df.shape[0]
cols = restaurant_df.shape[1]

count_empty_or_zero_cells = restaurant_df.isna().sum().sum()
total_elements = rows * cols

sparsity_of_matrix = count_empty_or_zero_cells/total_elements

print('Total Empty cells are : ', count_empty_or_zero_cells)
print('Total cells in Matrix are : ', total_elements)
print('Sparsity of Matrix are : ', sparsity_of_matrix)

Total Empty cells are :  999071
Total cells in Matrix are :  1019886
Sparsity of Matrix are :  0.9795908562329515


In [28]:
# 空值用 0 取代
restaurant_df.fillna(0, inplace = True)

In [29]:
restaurant_df.values

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [30]:
from scipy.sparse import csr_matrix
restaurant_df_matrix = csr_matrix(restaurant_df.values)

In [31]:
# 建立餘弦相似模型 (K-近鄰演算法)
from sklearn.neighbors import NearestNeighbors
nearest_neighbor_model = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
nearest_neighbor_model.fit(restaurant_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [32]:
# 隨機選擇一間餐廳
one_restaurant = restaurant_df.shape[0]
print('Total restaurants in our pivot table : ', one_restaurant)
random_restaurant_index = np.random.choice(one_restaurant)
print('Random restaurant Index : ', random_restaurant_index)

Total restaurants in our pivot table :  20814
Random restaurant Index :  15588


In [33]:
restaurant_df.iloc[random_restaurant_index]

clean_add_num_index
0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
6      0.0
7      0.0
8      0.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     0.0
29     0.0
30     0.0
31     0.0
32     0.0
33     0.0
34     0.0
35     0.0
36     0.0
37     0.0
38     0.0
39     0.0
40     0.0
41    42.0
42     0.0
43     0.0
44     0.0
45     0.0
46     0.0
47     0.0
48     0.0
Name: 立早餐, dtype: float64

In [34]:
# 向量顯示 random_book
one_dimensional_representation_of_restaurant_vector = restaurant_df.iloc[random_restaurant_index].values.reshape(1, -1)
one_dimensional_representation_of_restaurant_vector

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0., 42.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]])

In [35]:
# 相似餐廳與 random_restaurant 的餘弦距離指數
distances, indices = nearest_neighbor_model.kneighbors(one_dimensional_representation_of_restaurant_vector, n_neighbors = 6)
print('Distance :', distances)
print('Indices :', indices)

Distance : [[0. 0. 0. 0. 0. 0.]]
Indices : [[ 3414  8376  1496 20485 16025 10759]]


In [36]:
indices = indices.flatten()
distances = distances.flatten()
for i in range(0, len(indices)):
   
    if i == 0:
        print('Recommendations for {0}:\n'.format(restaurant_df.index[random_restaurant_index]))
    else:
        ## similar books
        print('{0}: {1}, with distance of {2}'.format(i, restaurant_df.index[indices[i]], distances[i]))

Recommendations for 立早餐:

1: 嘟好燒, with distance of 0.0
2: East23 日式料理, with distance of 0.0
3: 麵魂家, with distance of 0.0
4: 美大叔 / uncle made, with distance of 0.0
5: 張記蔥蛋堡, with distance of 0.0


In [37]:
# 找出全部餐廳的相似資料集 
my_dict = {}
for restaurant_index in range(restaurant_df.shape[0]):
    
    one_dimensional_representation_of_restaurant_vector = restaurant_df.iloc[restaurant_index].values.reshape(1, -1)
    distances, indices = nearest_neighbor_model.kneighbors(one_dimensional_representation_of_restaurant_vector, n_neighbors =4)
    indices = indices.flatten()
    distances = distances.flatten()
    similar_restaurants = []
    
    for i in range(0, len(indices)):
        
        if i == 0:
            original_restaurant = restaurant_df.index[restaurant_index]
        else:
            ## similar books
            similar_restaurants.append(restaurant_df.index[indices[i]])

        my_dict[original_restaurant] = similar_restaurants

In [41]:
# 秀出前3間相似
recommended_restaurant_df = pd.DataFrame(my_dict)
recommended_restaurant_df = recommended_restaurant_df.T
recommended_restaurant_df.columns = ['1st_Similar_restaurant', '2nd_Similar_restaurant', '3rd_Similar_restaurant']
recommended_restaurant_df.tail(200)

,1st_Similar_restaurant,2nd_Similar_restaurant,3rd_Similar_restaurant
黑米 cafe bistro,樂。野食,台北 藝奇新日本料理,好初Deli
黑米CAFE‧BISTR,Unicorn 法式甜點爆米花,UZNA OMOM 杏桃鬆餅屋－早餐,UNIQUE PASTA 維一 義大利餐廳
黑米Cafe Bistro,樂義 義大利餐廳 忠孝復興店,航情麵線總店,樂華夜市
黑米Café Bistro-ConfitRémi,香菇肉粥 鹹粥,《In Between牛排館》,La PASTA義麵屋 天母店
黑米Caf憿EBistro,Unicorn 法式甜點爆米花,UZNA OMOM 杏桃鬆餅屋－早餐,UNIQUE PASTA 維一 義大利餐廳
...,...,...,...
ＱＱ食堂,Que Pasa 幹嘛西班牙餐酒館,躼腳日式料理,法法小館
ＲＡＷ,樂。野食,台北 藝奇新日本料理,好初Deli
Ｗoobar,樂義 義大利餐廳 忠孝復興店,航情麵線總店,樂華夜市
ｋｉｋｉ ｔｈａｉ ｃａｆｅ忠孝店,樂。野食,台北 藝奇新日本料理,好初Deli


In [ ]:
# 餐廳推薦 
restaurant_name = '幸福食堂'
result = recommended_restaurant_df.loc[restaurant_name, : ]
print('I recommend you: \n')
for restaurants in list(result.values):
      print(restaurants)

In [ ]:
my_dict